# Defining tallies

We start by importing the `openmc` module, which contains the API to generate the input files for OpenMC.

In [2]:
import openmc
openmc.config['cross_sections'] = '/home/student/openmc_ncrystal_vm/endfb-viii.0-hdf5/cross_sections.xml'

One optional but very useful feature in Monte Carlo simulations is the definition of tallies. It is optional, because the simulation would run without them, but it is necessary in order to obtain anything from it. Tallies are specified using the `Tallies()` class, which generates the file `tallies.xml`.

The `Tallies()` object is a collection of `Tally()` objects that need to be defined independently.

In [2]:
tally1 = openmc.Tally(name='spectrum')
tally1.scores = ['flux']

The constructor takes as a parameter a name, which is useful to find the results later easier. Something we also need to define is the score in the `Tally.scores` attribute. We have to choose from:

- `flux`: total scalar flux. Units are particle-cm per source particle (track length). We need to divide by the volume of the region and multiply by the source intensity to obtain the flux.
- `current`: partial currents over a mesh surface, or net current over a cell surface (depending on which filters are applied, see below). `CellFrom()` and `Cell()` filters can also be used to find partial currents over cell surfaces. Units are particles per source particle. We need to divide by the surface area and multiply by the source intensity to find the actual current.
- `(n,gamma)`, `scatter`, `total`: compute the reaction rate for each reaction. Units are reactions per source particle. We need to divide by the volume of the region and multiply by the source intensity to obtain the reaction rate density.

A complete list of the scores is given in the [Specifying Tallies](https://docs.openmc.org/en/latest/usersguide/tallies.html) section of the User Guide.

## Filters

We need then to define which events will be counted, and how they will be segmented. This is given by *filters*, and given as an iterable in the `Tally.filters` attribute. These are possible filters:

- `CellFilter()`: filters events that happen inside of a given cell. Takes as a parameter a cell instance, a cell id, or an iterable of them. It can be used to construct the equivalent of an `F4` tally in MCNP or a `T-Track` tally in PHITS.
- `SurfaceFilter()`: filters events that happen on a given surface. Takes as a parameter a surface instance, a surface id, or an iterable of them. It can be used to construct the equivalent of an `F1` tally in MCNP or a `T-Cross` tally in PHITS.
- `MeshFilter()`: filters events that happen inside of elements of a mesh. Takes as a parameter a mesh object. It can be used to construct the equivalent of an `FMESH` tally in MCNP or a mesh `T-Track` tally in PHITS. 
- `EnergyFilter()`: filters events that happen in a given range of energy. Takes as a parameter a list of floats or numpy array that represent the energy group limits. 


A complete list is given in the [Constructing Tallies](https://docs.openmc.org/en/latest/pythonapi/base.html#constructing-tallies) section of the Python API.

## Example

Let's start defining the same example as in the previous section:

In [23]:
# Material definition

h2o = openmc.Material()
h2o.add_nuclide("H1", 2.0, "ao")
h2o.add_nuclide("O16", 1.0, "ao")
h2o.set_density("g/cm3", 1)
h2o.add_s_alpha_beta("c_H_in_H2O")

mats = openmc.Materials([h2o])
mats.export_to_xml()

# Geometry definition

sphere = openmc.Sphere(r=10, boundary_type="vacuum")
cell = openmc.Cell(region=-sphere, fill=h2o)
universe = openmc.Universe(cells=[cell])

geom = openmc.Geometry(universe)
geom.export_to_xml()

# Settings definition

source = openmc.Source()
source.space = openmc.stats.Point(xyz=(0.0, 0.0, 0.0))
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([1.0e6], [1.0])

settings = openmc.Settings()
settings.source = source
settings.run_mode = "fixed source"
settings.batches = 10
settings.particles = 10000
settings.export_to_xml()

We will now define two tallies:

- The absorption reaction rate inside of the cell.
- The total leakage from the moderator cell

In [24]:
tally1 = openmc.Tally(name='leakage')
tally1.scores=['current']
filter1 = openmc.SurfaceFilter(sphere)
tally1.filters = [filter1]

tally2 = openmc.Tally(name='absorption')
tally2.scores=['absorption']
filter2 = openmc.CellFilter(cell)
tally2.filters = [filter2]

tallies = openmc.Tallies([tally1, tally2])
tallies.export_to_xml()

In [25]:
!rm statepoint.*.h5
!rm summary.h5
openmc.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

We can see the result of the tally by reading the file `tallies.out`:

In [21]:
!cat tallies.out

 =======================>     TALLY 9: LEAKAGE     <========================

 Surface 4
   Total Material
     Current                              0.59727 +/- 0.00135303
 =====================>     TALLY 10: ABSORPTION     <======================

 Cell 4
   Total Material
     Absorption Rate                      0.400707 +/- 0.00128976


As we can see, the leakage gives the same value as in the summary table of the run (because the leakage from the moderator cell is the leakage from the whole simulation), and that the sum of the absorption and leakage add up to 1 source 

It is not very practical to read the `tallies.out`, particularly for postprocessing. For this, OpenMC provides a binary State Point file in standard HDF5 format. This contains the result of the tallies as well as restart the simulation.